https://www.geeksforgeeks.org/emotion-detection-using-convolutional-neural-networks-cnns/


In [ ]:
#Install required packages

!pip install keras
!pip install tensorflow
!pip install --upgrade keras tensorflow
!pip install --upgrade opencv-python

In [ ]:
#Import required packages
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten 
from keras.optimizers import Adam 
from keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.optimizers.schedules import ExponentialDecay 
import cv2 
from keras.models import model_from_json 
import numpy as np

In [ ]:
train_data_gen = ImageDataGenerator(rescale=1./255) 
validation_data_gen = ImageDataGenerator(rescale=1./255) 

In [ ]:
train_generator = train_data_gen.flow_from_directory( 
    'Train File Path', 
    target_size=(48, 48), 
    batch_size=64, 
    color_mode="grayscale", 
    class_mode='categorical') 

In [ ]:
validation_generator = validation_data_gen.flow_from_directory( 
        'Test file path', 
        target_size=(48, 48), 
        batch_size=64, 
        color_mode="grayscale", 
        class_mode='categorical') 

In [ ]:
emotion_model = Sequential() 
  
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',  
                         input_shape=(48, 48, 1))) 
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Dropout(0.25)) 
  
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) 
emotion_model.add(Dropout(0.25)) 
  
emotion_model.add(Flatten()) 
emotion_model.add(Dense(1024, activation='relu')) 
emotion_model.add(Dropout(0.5)) 
emotion_model.add(Dense(7, activation='softmax')) 
  
emotion_model.summary() 
  
cv2.ocl.setUseOpenCL(False) 
  
initial_learning_rate = 0.0001
lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=100000, decay_rate=0.96) 
  
optimizer = Adam(learning_rate=lr_schedule) 
  
emotion_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) 

In [ ]:
emotion_model_info = emotion_model.fit_generator( 
        train_generator, 
        steps_per_epoch=28709 // 64, 
        epochs=30, 
        validation_data=validation_generator, 
        validation_steps=7178 // 64) 

In [ ]:
emotion_model.evaluate(validation_generator)

In [ ]:
accuracy = emotion_model_info.history['accuracy'] 
val_accuracy = emotion_model_info.history['val_accuracy'] 
loss = emotion_model_info.history['loss'] 
val_loss = emotion_model_info.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt 
  
# Accuracy graph 
plt.subplot(1, 2, 1) 
plt.plot(accuracy, label='accuracy') 
plt.plot(val_accuracy, label='val accuracy') 
plt.title('Accuracy Graph') 
plt.xlabel('Epochs') 
plt.ylabel('Accuracy') 
plt.legend() 
  
# loss graph 
plt.subplot(1, 2, 2) 
plt.plot(loss, label='loss') 
plt.plot(val_loss, label='val loss') 
plt.title('Loss Graph') 
plt.xlabel('Epochs') 
plt.ylabel('Loss') 
plt.legend() 
  
plt.show() 

In [ ]:
model_json = emotion_model.to_json() 
with open("emotion_model.json", "w") as json_file: 
    json_file.write(model_json) 
  
# save trained model weight in .h5 file 
emotion_model.save_weights('emotion_model.h5') 